Sort people into leagues for curling
KL 8/30/2016

#This version starts with a random league and puts a random subset of people into each league;
updated to use new website export format

#REMEMBER: this will skip league managers for the moment

In [5]:
if True:
    %reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [6]:
setSeed = False

In [7]:
#set the random seed...if a number...will be able to recreate the random numbers
if setSeed == True:
    SEED = 42
elif setSeed == False:
    SEED = None

In [8]:
import pandas as pd
import numpy as np
import random
import math
import csv
import os.path

#used this to step into the function and debug it, also need line with Tracer()() 
from IPython.core.debugger import Tracer 

Define some functions up top

In [9]:
def addOne(leagueDict,oneLeague,oneName):
    if len(leagueDict[oneLeague])==0: #nothing there yet
        leagueDict[oneLeague] = oneName
    else:     #need to append
        leagueDict[oneLeague] = leagueDict[oneLeague].append(oneName)
    return leagueDict

In [10]:
def addManager(leagueDictionary,r,manager): #here r is the index in...
    useRow = r.index[0]
    oneName = regSorted.loc[useRow,['lastname', 'firstname']] 
    oneName = oneName.to_frame().transpose() #need this format to make nice output
    leagueDictionary[manager] = oneName
    regSorted.loc[useRow,manager]=np.nan
    regTracking.loc[useRow,manager]='yes'
    return leagueDictionary

Read in the data

In [11]:
#updated 8/30/2016 to use file from website
fName = 'BackupList_Registrations.csv' 
importInfo=pd.read_csv(fName)

In [12]:
#shuffle the information from the website site to match the format in the rest of this file
openLeagues = ['Sunday Night', 'Monday 4PM', 'Monday Night',
               'Tuesday 4PM','Wednesday 4PM','Thursday 4PM'] #updated to match the priority list
otherLeagues = ['League_4_Men', 'League_7_Ladies' ,'League_Scrod']
priorities = ['League_Pri_1','League_Pri_2','League_Pri_3','League_Pri_4','League_Pri_5','League_Pri_6']

regInfo = importInfo.loc[:,(['firstname','lastname','email','League_Number_Open'] + otherLeagues)]

for item in openLeagues:
    regInfo[item] = np.nan

nPeople = len(regInfo.index)
for idx in range(0,nPeople):
    for idxP in enumerate(priorities):
        #remember: enumerate will allow idxP[0] is the number, while idxP[1] is the value
        getOne = importInfo.loc[idx,idxP[1]]
        if getOne != 'None':
            regInfo.loc[idx,getOne] = idxP[0] + 1  #want 1,2,3 and not python zero index

In [13]:
#first, how many random numbers so I need?
nPeople = len(regInfo.index)
np.random.seed(SEED)
regInfo['randomNumCol'] = np.random.choice(range(nPeople), nPeople,replace = False)
##so now, moving back to the list of people, sort the list by the random number
#...still need to randomize people bc might have case where I can't grant first choice
regSorted = regInfo.sort_values(by = 'randomNumCol', ascending=True)
regSorted = regSorted.set_index(['randomNumCol'])

#copy so I can have an output tracking what was done
regTracking = regSorted.copy(deep=True)

setup the places to store the output

In [14]:
random.seed(SEED)
randLeagues = list(openLeagues) #make a copy of the list
random.shuffle(randLeagues) #does this in place, so don't make a new variable
randLeagues

['Thursday 4PM',
 'Monday 4PM',
 'Monday Night',
 'Tuesday 4PM',
 'Wednesday 4PM',
 'Sunday Night']

In [15]:
#might not want to randomize the leagues...changes outcome depending on how people prioritize

In [16]:
#want a way to quantify the most popular leagues...go back to regInfo
countFirst = {}
for name in openLeagues:
    #countFirst[name] = pd.DataFrame()
    countFirst[name] = len(regInfo.loc[regInfo[name]==1])

countFirst

#based on this...and the multiple runs, maybe should start with the least popular leagues in order
#to get the most people into their first choice
#though if we run Thursday 4 pm, Sunday, and Mondy 4 pm, have 1 person who doesn't even get 3rd choice

{'Monday 4PM': 13,
 'Monday Night': 21,
 'Sunday Night': 20,
 'Thursday 4PM': 20,
 'Tuesday 4PM': 14,
 'Wednesday 4PM': 12}

In [17]:
#setup the dictionary to hold the people information (start with openLeagues only)
forDictionary = openLeagues
#forDictionary.extend(otherLeagues)
leagueDict_open = {}
for name in forDictionary:
    leagueDict_open[name] = pd.DataFrame()

In [18]:
if True:
    #put the league managers into their league and set their the choice for that league to NaN
    manager = 'Sunday Night'
    r = regSorted.loc[(regSorted['lastname']=='Fragaszy') & (regSorted['firstname']=='Bill')]
    leagueDict_open = addManager(leagueDict_open,r,manager)

    manager = 'Monday 4PM'
    r = regSorted.loc[(regSorted['lastname']=='Brown') & (regSorted['firstname']=='Peter')]
    leagueDict_open = addManager(leagueDict_open,r,manager)

    manager = 'Monday Night'
    r = regSorted.loc[(regSorted['lastname']=='DiMassa') & (regSorted['firstname']=='Diane')]
    leagueDict_open = addManager(leagueDict_open,r,manager)

    manager = 'Tuesday 4PM'
    r = regSorted.loc[(regSorted['lastname']=='Thomson') & (regSorted['firstname']=='Kenny')]
    leagueDict_open = addManager(leagueDict_open,r,manager)

    manager = 'Wednesday 4PM'
    r = regSorted.loc[(regSorted['lastname']=='Murphy') & (regSorted['firstname']=='Katie')]
    leagueDict_open = addManager(leagueDict_open,r,manager)

    manager = 'Thursday 4PM'
    r = regSorted.loc[(regSorted['lastname']=='Gallagher') & (regSorted['firstname']=='Bill')]
    leagueDict_open = addManager(leagueDict_open,r,manager)

In [19]:
#set the limit for each league:
sevenTeams = 20 #change to 28 (remember league manager already in here)
fourteenTeams = 20

d = {'Sunday Night': sevenTeams,
     'Monday 4PM': sevenTeams,
     'Monday Night': fourteenTeams,
     'Tuesday 4PM': sevenTeams,
     'Wednesday 4PM': sevenTeams,
     'Thursday 4PM': sevenTeams,}

nLimit = pd.DataFrame({'nLimit' : d})
del d

In [20]:
##NEED better names for the issues

In [21]:
issueNames = ['notInFirst','notInSecond','notInThird','noSecondChoice','noAdditionalChoice',
              'notInNext','notInNext2']

#setup the dictionary to hold the issues
trackIssues = {}
for name in issueNames:
    trackIssues[name] = pd.DataFrame()

In [22]:
pd.options.mode.chained_assignment = None  # default='warn'

In [23]:
#change tacks...go through a league and grant first choices...don't worry about 2nd choices yet

In [24]:
for oneLeague in randLeagues:
    subset = regSorted.loc[regSorted[oneLeague]==1]
    subset.reset_index(inplace = True)
    nPeople = len(subset.index)
    
    #what is the limit for this league?
    oneLimit = nLimit.loc[oneLeague][0]
    
    for idx in range(0,nPeople):
        getIdx = subset.loc[idx,'randomNumCol']
        if idx < oneLimit:           
            oneName = subset.loc[idx,['lastname', 'firstname']] 
            oneName = oneName.to_frame().transpose() #need this format to make nice output 
            leagueDict_open = addOne(leagueDict_open,oneLeague,oneName)
            
            regTracking.loc[getIdx,oneLeague] = 'yes'
            regSorted.loc[getIdx,oneLeague] = np.nan
        elif idx >= oneLimit:
            oneName = subset.loc[idx,['lastname', 'firstname']] 
            oneName = oneName.to_frame().transpose() #need this format to make nice output 
            trackIssues = addOne(trackIssues, 'notInFirst',oneName)
            
            regTracking.loc[getIdx,oneLeague] = 'no1'
            regSorted.loc[getIdx,oneLeague] = np.nan

In [25]:
#OK...regTracking now has a 'no' for those people who did not get their first choice...use that to
#put those people into their second choice league
for oneLeague in randLeagues:    
    #how many spaces are left in this league? Use to figure out how many to add
    nRemaining = nLimit.loc[oneLeague][0] - len(leagueDict_open[oneLeague])  
    print oneLeague, nRemaining

Thursday 4PM 0
Monday 4PM 7
Monday Night -1
Tuesday 4PM 6
Wednesday 4PM 7
Sunday Night 0


In [26]:
#go through the list of people who did not get their 1st choice league, and see about
#putting them into their second choice league
for idx in range(0,len(regTracking)):  
    doSomething = []
    r = regTracking.loc[idx,openLeagues]
    for item in range(0,len(r)):
        if r[item] == 'no1':
            doSomething = True

    if doSomething:
        oneName = regTracking.loc[idx,['lastname', 'firstname']] 
        oneName = oneName.to_frame().transpose() #need this format to make nice output 

        #get their next choice
        newPriority = regSorted.loc[idx,openLeagues].dropna().min()

        if math.isnan(newPriority) == False: #have another league listed
            league2 = regSorted.loc[idx,openLeagues].dropna().idxmin() #returns name of league
            #first need to check if the league has space
            if len(leagueDict_open[league2]) < (nLimit.loc[league2,'nLimit']):
                leagueDict_open = addOne(leagueDict_open,league2,oneName)
                regTracking.loc[idx,league2] = 'yes'
                regSorted.loc[idx,league2] = np.nan
            else:
                #no space in second choice league
                trackIssues = addOne(trackIssues,'notInSecond',oneName)
                #set their first choice to 'no' and move onto second choices
                regTracking.loc[idx,league2] = 'no1' 
                regSorted.loc[idx,league2] = np.nan
                print oneName, 'no space in second choice; would need third'

        elif math.isnan(newPriority):
            trackIssues = addOne(trackIssues,'noSecondChoice',oneName)

In [27]:
#what about those people who did not get their second choice?
for idx in range(0,len(regTracking)):  
    doSomething = 0
    r = regTracking.loc[idx,openLeagues]
    for item in range(0,len(r)):
        if r[item] == 'no1': #need two 'no' responses to doSomething
             doSomething = doSomething+1

    if doSomething > 1:
        oneName = regTracking.loc[idx,['lastname', 'firstname']] 
        oneName = oneName.to_frame().transpose() #need this format to make nice output 

        #get their next choice
        newPriority = regSorted.loc[idx,openLeagues].dropna().min()

        if math.isnan(newPriority) == False: #have another league listed
            league2 = regSorted.loc[idx,openLeagues].dropna().idxmin() #returns name of league
            #first need to check if the league has space
            if len(leagueDict_open[league2]) < (nLimit.loc[league2,'nLimit']):
                leagueDict_open = addOne(leagueDict_open,league2,oneName)
                regTracking.loc[idx,league2] = 'yes'
                regSorted.loc[idx,league2] = np.nan
            else:
                #no space in second choice league
                trackIssues = addOne(trackIssues,'notInThird',oneName)
                #set their first choice to 'no' and move onto second choices
                regTracking.loc[idx,league2] = 'no1' 
                regSorted.loc[idx,league2] = np.nan
                print oneName, 'no space in third choice; would need fourth'

        elif math.isnan(newPriority):
            trackIssues = addOne(trackIssues,'noAdditionalChoice',oneName)

In [28]:
if len(trackIssues['notInThird'])==0:
    #no longer trying to grant a first choice
    print 'yes'

yes


In [29]:
#this time...work from the bottom up...only need to work on someone if nWanted > 1

In [30]:
for idx in reversed(range(0,len(regTracking))):
    nLeagues = regTracking.loc[idx,'League_Number_Open']
    if nLeagues != 'None' : 
        if int(nLeagues) > 1:   
            oneName = regTracking.loc[idx,['lastname', 'firstname']] 
            oneName = oneName.to_frame().transpose() #need this format to make nice output 

            nextPriority = regSorted.loc[idx,openLeagues].dropna().min()
            if math.isnan(nextPriority) == False: #have another league listed
                nextLeague = regSorted.loc[idx,openLeagues].dropna().idxmin()
                #first need to check if the league has space
                if len(leagueDict_open[nextLeague]) < (nLimit.loc[nextLeague,'nLimit']):
                    leagueDict_open = addOne(leagueDict_open,nextLeague,oneName)
                    regTracking.loc[idx,nextLeague] = 'yes'
                    regSorted.loc[idx,nextLeague] = np.nan
                else:
                    #no space in second choice league
                    trackIssues = addOne(trackIssues,'notInSecond',oneName)
                    #set their first choice to 'no' and move onto second choices
                    regTracking.loc[idx,nextLeague] = 'no2' 
                    regSorted.loc[idx,nextLeague] = np.nan
                    print oneName, 'no space in next choice; would need another'

            elif math.isnan(newPriority):
                trackIssues = addOne(trackIssues,'noAdditionalChoice',oneName)

     lastname firstname
111  Thunberg      Eric no space in next choice; would need another
     lastname firstname
108  Voorhees   Kenneth no space in next choice; would need another
    lastname firstname
103    Brown     Peter no space in next choice; would need another
   lastname firstname
85  Baggett    Walter no space in next choice; would need another
   lastname firstname
75  BAYBUTT    PHILIP no space in next choice; would need another
   lastname firstname
64   Verner     Susan no space in next choice; would need another
   lastname firstname
57   Hannon      Jane no space in next choice; would need another
    lastname firstname
56  Fragaszy      Bill no space in next choice; would need another
   lastname     firstname
50  Pilotte  Courtney Lee no space in next choice; would need another
   lastname firstname
46  Perella      Anne no space in next choice; would need another
   lastname firstname
32   Hannon      Neal no space in next choice; would need another
   lastname 

NameError: name 'newPriority' is not defined

In [ ]:
#what about those people who are still looking for a second league?
for idx in reversed(range(0,len(regTracking))):  
    nLeagues = regTracking.loc[idx,'League_Number_Open']
    if nLeagues != 'None':
        doSomething = 0
        r = regTracking.loc[idx,openLeagues]
        for item in range(0,len(r)):
            if r[item] == 'no2': #need two 'no' responses to doSomething
                 doSomething = doSomething+1
       
        if (doSomething > 0) & (int(nLeagues) > 1):
            oneName = regTracking.loc[idx,['lastname', 'firstname']] 
            oneName = oneName.to_frame().transpose() #need this format to make nice output 
            #get their next choice
            newPriority = regSorted.loc[idx,openLeagues].dropna().min()

            if math.isnan(newPriority) == False: #have another league listed
                league2 = regSorted.loc[idx,openLeagues].dropna().idxmin() #returns name of league
                #first need to check if the league has space
                if len(leagueDict_open[league2]) < (nLimit.loc[league2,'nLimit']):
                    leagueDict_open = addOne(leagueDict_open,league2,oneName)
                    regTracking.loc[idx,league2] = 'yes'
                    regSorted.loc[idx,league2] = np.nan
                else:
                    #no space in league
                    trackIssues = addOne(trackIssues,'notInNext',oneName) 
                    #set their first choice to 'no' and move onto second choices
                    regTracking.loc[idx,league2] = 'no2' 
                    regSorted.loc[idx,league2] = np.nan
                    print oneName, 'no space in next choice; would need fourth'

            elif math.isnan(newPriority):
                trackIssues = addOne(trackIssues,'noAdditionalChoice',oneName)

In [ ]:
for idx in reversed(range(0,len(regTracking))):  
    nLeagues = regTracking.loc[idx,'League_Number_Open']
    if nLeagues != 'None':
        doSomething = 0
        r = regTracking.loc[idx,openLeagues]
        for item in range(0,len(r)):
            if r[item] == 'no2': #need two 'no' responses to doSomething
                 doSomething = doSomething+1
       
        if (doSomething > 1) & (int(nLeagues) > 1):
            oneName = regTracking.loc[idx,['lastname', 'firstname']] 
            oneName = oneName.to_frame().transpose() #need this format to make nice output 
            #get their next choice
            newPriority = regSorted.loc[idx,openLeagues].dropna().min()

            if math.isnan(newPriority) == False: #have another league listed
                league2 = regSorted.loc[idx,openLeagues].dropna().idxmin() #returns name of league
                #first need to check if the league has space
                if len(leagueDict_open[league2]) < (nLimit.loc[league2,'nLimit']):
                    leagueDict_open = addOne(leagueDict_open,league2,oneName)
                    regTracking.loc[idx,league2] = 'yes'
                    regSorted.loc[idx,league2] = np.nan
                else:
                    #no space in league
                    trackIssues = addOne(trackIssues,'notInNext2',oneName) 
                    #set their first choice to 'no' and move onto second choices
                    regTracking.loc[idx,league2] = 'no2' 
                    regSorted.loc[idx,league2] = np.nan
                    print oneName, 'no space in next choice; would need fourth'

            elif math.isnan(newPriority):
                trackIssues = addOne(trackIssues,'noAdditionalChoice',oneName)

In [ ]:
#OK...regTracking now has a 'no' for those people who did not get their first choice...use that to
#put those people into their second choice league
for oneLeague in randLeagues:    
    #how many spaces are left in this league? Use to figure out how many to add
    nRemaining = nLimit.loc[oneLeague][0] - len(leagueDict_open[oneLeague])  
    print oneLeague, nRemaining

In [ ]:
#make a new dictionary for other leagues...concat with openLeagues when done
# do doubles and competitive league by hand...
otherLeagues = ['League_4_Men', 'League_7_Ladies' ,'League_Scrod']

leagueDict_other = {}
for name in otherLeagues:
    leagueDict_other[name] = pd.DataFrame()

In [ ]:
if False:
    #put league managers in for other
    manager = 'League_4_Men'
    r = regSorted.loc[(regSorted['lastname']=='Pijanowski') & (regSorted['firstname']=='Brian')]
    leagueDict_other = addManager(leagueDict_other,r,manager)

    manager = 'League_7_Ladies'
    r = regSorted.loc[(regSorted['lastname']=='Galligan') & (regSorted['firstname']=='Natalie')]
    leagueDict_other = addManager(leagueDict_other,r,manager)

    manager = 'League_Scrod'
    r = regSorted.loc[(regSorted['lastname']=='Pilotte') & (regSorted['firstname']=='Courtney Lee')]
    leagueDict_other = addManager(leagueDict_other,r,manager)

In [ ]:
#only have to go through once
for idx in range(0,nPeople):
    for oL in otherLeagues:    
        oneName = regSorted.loc[idx,['lastname', 'firstname']] 
        oneName = oneName.to_frame().transpose() #need this format to make nice output
        if regSorted.loc[idx,oL]=='Yes': #careful with trailing spaces
            leagueDict_other = addOne(leagueDict_other,oL,oneName)
            #now that I have added the person to a league, change their priority to NaN
            regTracking.loc[idx,oL] = 'yes'
            regSorted.loc[idx,oL] = np.nan

In [ ]:
#combine the two dictionaries
leagueDict_all = leagueDict_open.copy()
leagueDict_all.update(leagueDict_other)

#combine the list of names
allLeagues = list(openLeagues)
allLeagues.extend(otherLeagues)

In [ ]:
#export the results to a series of CSV files (CAREFUL...will delete existing files)
for fn in allLeagues:
    fName = '%s.csv' % fn
    #check if the file exists...delete if it does
    if os.path.isfile(fName):
        os.remove(fName)
        
    leagueDict_all[fn].to_csv(fName)

In [ ]:
#lets try some plotting
%matplotlib inline

In [ ]:
tData = regInfo.loc[:,openLeagues]
tData.count().plot(kind = 'bar')

In [ ]:
regInfo.to_csv('regOut.csv')

In [ ]:
tData.groupby(by=openLeagues)

In [ ]:
tData.get_value()

In [ ]:
tData.

In [ ]:
_values()

In [ ]:
regInfo.loc[:,'Sunday Night'].plot(kind = 'bar')

In [ ]:
tData = regInfo.loc[:,'Sunday Night']

In [ ]:
tData.value_counts()

In [ ]:
tData.value_counts().plot(kind = 'bar')

In [ ]:
for problem in issueNames:
    print problem, ': ', len(trackIssues[problem]), ' people in this category'
    
randLeagues

In [ ]:
for problem in issueNames:
    print problem, ': ', len(trackIssues[problem]), ' people in this category'
    
randLeagues

In [529]:
regInfo.to_csv('sorted.csv')

In [351]:
#ran multiple times...order the leagues are randomized will change the output

In [353]:
for problem in issueNames:
    print problem, ': ', len(trackIssues[problem]), ' people in this category'
    
randLeagues

notInFirst :  0  people in this category
notInSecond :  5  people in this category
notInThird :  0  people in this category
noSecondChoice :  16  people in this category
noAdditionalChoice :  57  people in this category
notInNext :  0  people in this category
notInNext2 :  0  people in this category


['Monday Night',
 'Thursday 4PM',
 'Wednesday 4PM',
 'Monday 4PM',
 'Sunday Night',
 'Tuesday 4PM']

In [354]:
for problem in issueNames:
    print problem, ': ', len(trackIssues[problem]), ' people in this category'
    
randLeagues

notInFirst :  0  people in this category
notInSecond :  5  people in this category
notInThird :  0  people in this category
noSecondChoice :  16  people in this category
noAdditionalChoice :  57  people in this category
notInNext :  0  people in this category
notInNext2 :  0  people in this category


['Monday Night',
 'Thursday 4PM',
 'Wednesday 4PM',
 'Monday 4PM',
 'Sunday Night',
 'Tuesday 4PM']

In [355]:
for problem in issueNames:
    print problem, ': ', len(trackIssues[problem]), ' people in this category'

randLeagues

notInFirst :  0  people in this category
notInSecond :  5  people in this category
notInThird :  0  people in this category
noSecondChoice :  16  people in this category
noAdditionalChoice :  57  people in this category
notInNext :  0  people in this category
notInNext2 :  0  people in this category


['Monday Night',
 'Thursday 4PM',
 'Wednesday 4PM',
 'Monday 4PM',
 'Sunday Night',
 'Tuesday 4PM']